### ### SETUP

In [26]:
from ragas.metrics import faithfulness, answer_relevancy
print("metrics loaded")

metrics loaded


C:\Users\user\AppData\Local\Temp\ipykernel_5024\1565457826.py:1: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import faithfulness, answer_relevancy
C:\Users\user\AppData\Local\Temp\ipykernel_5024\1565457826.py:1: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import faithfulness, answer_relevancy


In [27]:
%pip install langchain-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="gemma3:4b",
    temperature=0
)


EVALUATOR_LLM = LangchainLLMWrapper(llm)

C:\Users\user\AppData\Local\Temp\ipykernel_5024\135793725.py:9: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  EVALUATOR_LLM = LangchainLLMWrapper(llm)


Recursion is a programming technique where a function calls itself within its own definition to solve a smaller version of the same problem until a simple base case is reached, at which point the solutions are combined back up the chain of calls.


### WRITE TEST TO CHECK MY LOCAL LLM RESPONSE


In [35]:

from ragas.llms import LangchainLLMWrapper
from ragas import evaluate
from ragas.metrics import LLMContextRecall
from datasets import Dataset

from datasets import Dataset

data = {
    "question": [
        "Who is the current president of the United States?"
    ],
    "answer": [
        "Joe Biden"
    ],
    "contexts": [[
        "Joe Biden serves as the current president of America"
    ]],
    "ground_truth": [
        "Joe Biden serves as the current president of America"
    ]
}

dataset = Dataset.from_dict(data)

result = evaluate(
    dataset=dataset,
    metrics=[LLMContextRecall()],
    llm=EVALUATOR_LLM,
)
result


C:\Users\user\AppData\Local\Temp\ipykernel_5024\1878386644.py:3: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

{'context_recall': 1.0000}

### USE SAME DATA FOR CONTEXT PRECISION


In [33]:
from ragas.metrics import ContextPrecision
data = {
    "question": [
        "Who is the current president of the United States?"
    ],
    "answer": [
        "Joe Biden"
    ],
    "contexts": [[
        "Joe Biden serves as the current president of the United States.",
        "The Eiffel Tower is in Paris.",
        "Bananas are berries.",
        "The moon has no atmosphere."
    ]],
    "ground_truth": [
        "Joe Biden"
    ]
}

dataset = Dataset.from_dict(data)
context_relevancy = ContextPrecision(llm=EVALUATOR_LLM)

result = evaluate(
    dataset=dataset,
    metrics=[context_relevancy],
)

result.to_pandas()


C:\Users\user\AppData\Local\Temp\ipykernel_5024\3973006881.py:1: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import ContextPrecision


Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

,user_input,retrieved_contexts,response,reference,context_precision
0,Who is the current president of the United Sta...,[Joe Biden serves as the current president of ...,Joe Biden,Joe Biden,1.0
